# Config

In [1]:
import config
from langchain_openai import OpenAIEmbeddings
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction
model = "text-embedding-ada-002"
openai_api_key = config.OPEN_AI_KEY

D:\Programming\Code-Projects\business_agent\pythonProject\.venv\Lib\site-packages\onnxruntime\capi\onnxruntime_validation.py:26: UserWarning: Unsupported Windows version (11). ONNX Runtime supports Windows 10 and above, only.
  warnings.warn(


# Embedding definition

In [2]:
embeddings = OpenAIEmbeddings(
    model=model,
    openai_api_key=openai_api_key
)
embedding_function = OpenAIEmbeddingFunction(api_key=openai_api_key, model_name=model)


# Embedding responses

In [3]:
import pandas as pd


responses = pd.read_csv("responses.csv")
responses["embedding"] = responses["text"].apply(lambda x: embeddings.embed_query(x))
responses.to_csv("responses_with_embeddings.csv", index=False)

# Create Connection

In [4]:
import chromadb
import warnings
warnings.filterwarnings("ignore")

client = chromadb.Client()
client.heartbeat()

1709931420422598500

In [5]:
from ast import literal_eval
responses_with_embeddings = pd.read_csv("responses_with_embeddings.csv")
responses_with_embeddings["embedding"] = responses_with_embeddings["embedding"].apply(literal_eval)
responses_with_embeddings["id"] = responses_with_embeddings["id"].apply(str)
collection = client.get_or_create_collection("business_with_text",embedding_function=embedding_function)
for index, row in responses_with_embeddings.iterrows():
    collection.add(
        ids=responses_with_embeddings["id"].tolist(),
        embeddings=responses_with_embeddings["embedding"].tolist(),
        documents=responses_with_embeddings["text"].tolist()
    )


Add of existing embedding ID: 1
Add of existing embedding ID: 2
Add of existing embedding ID: 3
Add of existing embedding ID: 4
Add of existing embedding ID: 5
Add of existing embedding ID: 6
Add of existing embedding ID: 7
Add of existing embedding ID: 8
Insert of existing embedding ID: 1
Insert of existing embedding ID: 2
Insert of existing embedding ID: 3
Insert of existing embedding ID: 4
Insert of existing embedding ID: 5
Insert of existing embedding ID: 6
Insert of existing embedding ID: 7
Insert of existing embedding ID: 8
Add of existing embedding ID: 1
Add of existing embedding ID: 2
Add of existing embedding ID: 3
Add of existing embedding ID: 4
Add of existing embedding ID: 5
Add of existing embedding ID: 6
Add of existing embedding ID: 7
Add of existing embedding ID: 8
Insert of existing embedding ID: 1
Insert of existing embedding ID: 2
Insert of existing embedding ID: 3
Insert of existing embedding ID: 4
Insert of existing embedding ID: 5
Insert of existing embedding ID: 

In [6]:
def query(texto):
    query_embedding = embeddings.embed_query(texto)
    results = collection.query(query_embeddings=query_embedding, n_results=3, include=['distances','documents'])
    df = pd.DataFrame({
                    'id':results['ids'][0], 
                    'score':results['distances'][0],
                    'text': results['documents'][0],
                    })
    
    return df

In [7]:
import gradio as gr
with gr.Blocks() as demo:
    busqueda = gr.Textbox(label="Search")
    output = gr.DataFrame(headers=['text'])
    greet_btn = gr.Button("Ask")
    greet_btn.click(fn=query, inputs=[busqueda], outputs=output)

demo.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
